In [1]:
from pyspark.sql import SparkSession 
sc = SparkSession.builder.master("local[*]").getOrCreate()


In [2]:
data = sc.read.csv("/FileStore/tables/Bike_Rental_UCI_dataset-bb6c6.csv",inferSchema=True,header=True)

In [3]:
data.show()

+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+
season| yr|mnth| hr|holiday|workingday|weathersit|temp| hum|windspeed|dayOfWeek|days|demand|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+
 1| 0| 1| 0| 0| 0| 1|0.24|0.81| 0.0| Sat| 0| 16|
 1| 0| 1| 1| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 40|
 1| 0| 1| 2| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 32|
 1| 0| 1| 3| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 13|
 1| 0| 1| 4| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 1|
 1| 0| 1| 5| 0| 0| 2|0.24|0.75| 0.0896| Sat| 0| 1|
 1| 0| 1| 6| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 2|
 1| 0| 1| 7| 0| 0| 1| 0.2|0.86| 0.0| Sat| 0| 3|
 1| 0| 1| 8| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 8|
 1| 0| 1| 9| 0| 0| 1|0.32|0.76| 0.0| Sat| 0| 14|
 1| 0| 1| 10| 0| 0| 1|0.38|0.76| 0.2537| Sat| 0| 36|
 1| 0| 1| 11| 0| 0| 1|0.36|0.81| 0.2836| Sat| 0| 56|
 1| 0| 1| 12| 0| 0| 1|0.42|0.77| 0.2836| Sat| 0| 84|
 1| 0| 1| 13| 0| 0| 2|0.46|0.72| 0.2985| Sat| 0| 94|
 1| 0| 1| 14| 0| 0| 2|0.46|0.72| 0.2836| Sat| 0| 106|
 1| 0| 1| 15| 0| 0| 2|0.44|0.77| 0.2985| Sat| 0| 110|
 1| 0| 1| 16| 0| 0| 2|0.42|0.82| 0.2985| Sat| 0| 93|
 1| 0| 1| 17| 0| 0| 2|0.44|0.82| 0.2836| Sat| 0| 67|
 1| 0| 1| 18| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 35|
 1| 0| 1| 19| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 37|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+
only showing top 20 rows

In [4]:
from pyspark.ml.classification import (LogisticRegression,DecisionTreeClassifier,RandomForestClassifier)
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

In [5]:
indexer = StringIndexer(inputCol="dayOfWeek",outputCol="day_cat")

In [6]:
indexed_data = indexer.fit(data).transform(data)
indexed_data.show()

+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+
season| yr|mnth| hr|holiday|workingday|weathersit|temp| hum|windspeed|dayOfWeek|days|demand|day_cat|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+
 1| 0| 1| 0| 0| 0| 1|0.24|0.81| 0.0| Sat| 0| 16| 0.0|
 1| 0| 1| 1| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 40| 0.0|
 1| 0| 1| 2| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 32| 0.0|
 1| 0| 1| 3| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 13| 0.0|
 1| 0| 1| 4| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 1| 0.0|
 1| 0| 1| 5| 0| 0| 2|0.24|0.75| 0.0896| Sat| 0| 1| 0.0|
 1| 0| 1| 6| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 2| 0.0|
 1| 0| 1| 7| 0| 0| 1| 0.2|0.86| 0.0| Sat| 0| 3| 0.0|
 1| 0| 1| 8| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 8| 0.0|
 1| 0| 1| 9| 0| 0| 1|0.32|0.76| 0.0| Sat| 0| 14| 0.0|
 1| 0| 1| 10| 0| 0| 1|0.38|0.76| 0.2537| Sat| 0| 36| 0.0|
 1| 0| 1| 11| 0| 0| 1|0.36|0.81| 0.2836| Sat| 0| 56| 0.0|
 1| 0| 1| 12| 0| 0| 1|0.42|0.77| 0.2836| Sat| 0| 84| 0.0|
 1| 0| 1| 13| 0| 0| 2|0.46|0.72| 0.2985| Sat| 0| 94| 0.0|
 1| 0| 1| 14| 0| 0| 2|0.46|0.72| 0.2836| Sat| 0| 106| 0.0|
 1| 0| 1| 15| 0| 0| 2|0.44|0.77| 0.2985| Sat| 0| 110| 0.0|
 1| 0| 1| 16| 0| 0| 2|0.42|0.82| 0.2985| Sat| 0| 93| 0.0|
 1| 0| 1| 17| 0| 0| 2|0.44|0.82| 0.2836| Sat| 0| 67| 0.0|
 1| 0| 1| 18| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 35| 0.0|
 1| 0| 1| 19| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 37| 0.0|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+
only showing top 20 rows

In [7]:
indexed_data.select('day_cat').distinct().orderBy('day_cat').show()

+-------+
day_cat|
+-------+
 0.0|
 1.0|
 2.0|
 3.0|
 4.0|
 5.0|
 6.0|
+-------+

In [8]:
from pyspark.ml.feature import VectorAssembler

vec = VectorAssembler(inputCols=['season','yr','mnth','hr','holiday','workingday','weathersit','days','day_cat'],outputCol='features')

In [9]:
data1 = vec.transform(indexed_data)

In [10]:
data1.show()

+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+--------------------+
season| yr|mnth| hr|holiday|workingday|weathersit|temp| hum|windspeed|dayOfWeek|days|demand|day_cat| features|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+--------------------+
 1| 0| 1| 0| 0| 0| 1|0.24|0.81| 0.0| Sat| 0| 16| 0.0|(9,[0,2,6],[1.0,1...|
 1| 0| 1| 1| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 40| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 2| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 32| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 3| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 13| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 4| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 1| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 5| 0| 0| 2|0.24|0.75| 0.0896| Sat| 0| 1| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 6| 0| 0| 1|0.22| 0.8| 0.0| Sat| 0| 2| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 7| 0| 0| 1| 0.2|0.86| 0.0| Sat| 0| 3| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 8| 0| 0| 1|0.24|0.75| 0.0| Sat| 0| 8| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 9| 0| 0| 1|0.32|0.76| 0.0| Sat| 0| 14| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 10| 0| 0| 1|0.38|0.76| 0.2537| Sat| 0| 36| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 11| 0| 0| 1|0.36|0.81| 0.2836| Sat| 0| 56| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 12| 0| 0| 1|0.42|0.77| 0.2836| Sat| 0| 84| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 13| 0| 0| 2|0.46|0.72| 0.2985| Sat| 0| 94| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 14| 0| 0| 2|0.46|0.72| 0.2836| Sat| 0| 106| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 15| 0| 0| 2|0.44|0.77| 0.2985| Sat| 0| 110| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 16| 0| 0| 2|0.42|0.82| 0.2985| Sat| 0| 93| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 17| 0| 0| 2|0.44|0.82| 0.2836| Sat| 0| 67| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 18| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 35| 0.0|(9,[0,2,3,6],[1.0...|
 1| 0| 1| 19| 0| 0| 3|0.42|0.88| 0.2537| Sat| 0| 37| 0.0|(9,[0,2,3,6],[1.0...|
+------+---+----+---+-------+----------+----------+----+----+---------+---------+----+------+-------+--------------------+
only showing top 20 rows

In [11]:
modelData = data1.select('features','demand')

In [12]:
modelData.describe().show()

+-------+------------------+
summary| demand|
+-------+------------------+
 count| 17379|
 mean|189.46308763450142|
 stddev| 181.3875990918646|
 min| 1|
 max| 977|
+-------+------------------+

In [13]:
modelData.show(truncate=False)

+--------------------------------+------+
features |demand|
+--------------------------------+------+
(9,[0,2,6],[1.0,1.0,1.0]) |16 |
(9,[0,2,3,6],[1.0,1.0,1.0,1.0]) |40 |
(9,[0,2,3,6],[1.0,1.0,2.0,1.0]) |32 |
(9,[0,2,3,6],[1.0,1.0,3.0,1.0]) |13 |
(9,[0,2,3,6],[1.0,1.0,4.0,1.0]) |1 |
(9,[0,2,3,6],[1.0,1.0,5.0,2.0]) |1 |
(9,[0,2,3,6],[1.0,1.0,6.0,1.0]) |2 |
(9,[0,2,3,6],[1.0,1.0,7.0,1.0]) |3 |
(9,[0,2,3,6],[1.0,1.0,8.0,1.0]) |8 |
(9,[0,2,3,6],[1.0,1.0,9.0,1.0]) |14 |
(9,[0,2,3,6],[1.0,1.0,10.0,1.0])|36 |
(9,[0,2,3,6],[1.0,1.0,11.0,1.0])|56 |
(9,[0,2,3,6],[1.0,1.0,12.0,1.0])|84 |
(9,[0,2,3,6],[1.0,1.0,13.0,2.0])|94 |
(9,[0,2,3,6],[1.0,1.0,14.0,2.0])|106 |
(9,[0,2,3,6],[1.0,1.0,15.0,2.0])|110 |
(9,[0,2,3,6],[1.0,1.0,16.0,2.0])|93 |
(9,[0,2,3,6],[1.0,1.0,17.0,2.0])|67 |
(9,[0,2,3,6],[1.0,1.0,18.0,3.0])|35 |
(9,[0,2,3,6],[1.0,1.0,19.0,3.0])|37 |
+--------------------------------+------+
only showing top 20 rows

In [14]:
trainData,testData = modelData.randomSplit([0.7,0.3])

In [15]:
trainData = trainData.withColumnRenamed(('demand'),('label'))

In [16]:
testData = testData.withColumnRenamed(('demand'), ('label'))

In [17]:
trainData.show(truncate=False)

+--------------------------------+-----+
features |label|
+--------------------------------+-----+
(9,[0,2,3,6],[1.0,1.0,1.0,1.0]) |40 |
(9,[0,2,3,6],[1.0,1.0,2.0,1.0]) |32 |
(9,[0,2,3,6],[1.0,1.0,5.0,2.0]) |1 |
(9,[0,2,3,6],[1.0,1.0,6.0,1.0]) |2 |
(9,[0,2,3,6],[1.0,1.0,10.0,1.0])|36 |
(9,[0,2,3,6],[1.0,1.0,11.0,1.0])|56 |
(9,[0,2,3,6],[1.0,1.0,12.0,1.0])|84 |
(9,[0,2,3,6],[1.0,1.0,14.0,2.0])|106 |
(9,[0,2,3,6],[1.0,1.0,15.0,2.0])|110 |
(9,[0,2,3,6],[1.0,1.0,16.0,2.0])|93 |
(9,[0,2,3,6],[1.0,1.0,17.0,2.0])|67 |
(9,[0,2,3,6],[1.0,1.0,19.0,3.0])|37 |
(9,[0,2,3,6],[1.0,1.0,20.0,2.0])|36 |
(9,[0,2,3,6],[1.0,1.0,21.0,2.0])|34 |
(9,[0,2,3,6],[1.0,1.0,22.0,2.0])|28 |
(9,[0,2,6],[1.0,1.0,1.0]) |16 |
(9,[0,2,6,7],[1.0,1.0,1.0,13.0])|28 |
(9,[0,2,6,7],[1.0,1.0,1.0,19.0])|13 |
(9,[0,2,6,7],[1.0,1.0,1.0,25.0])|28 |
(9,[0,2,6,7],[1.0,1.0,2.0,6.0]) |25 |
+--------------------------------+-----+
only showing top 20 rows

In [18]:
testData.show(truncate=False)

+----------------------------------+-----+
features |label|
+----------------------------------+-----+
(9,[0,2,3,6],[1.0,1.0,3.0,1.0]) |13 |
(9,[0,2,3,6],[1.0,1.0,4.0,1.0]) |1 |
(9,[0,2,3,6],[1.0,1.0,7.0,1.0]) |3 |
(9,[0,2,3,6],[1.0,1.0,8.0,1.0]) |8 |
(9,[0,2,3,6],[1.0,1.0,9.0,1.0]) |14 |
(9,[0,2,3,6],[1.0,1.0,13.0,2.0]) |94 |
(9,[0,2,3,6],[1.0,1.0,18.0,3.0]) |35 |
(9,[0,2,3,6],[1.0,1.0,23.0,2.0]) |39 |
(9,[0,2,6,7],[1.0,2.0,1.0,46.0]) |29 |
(9,[0,2,6,7],[1.0,3.0,2.0,73.0]) |76 |
(9,[0,2,6,7],[1.0,12.0,2.0,359.0])|44 |
(9,[0,2,6,7],[2.0,4.0,2.0,101.0]) |43 |
(9,[0,2,6,7],[2.0,5.0,1.0,136.0]) |98 |
(9,[0,2,6,7],[2.0,6.0,1.0,157.0]) |117 |
(9,[0,2,6,7],[3.0,6.0,1.0,171.0]) |116 |
(9,[0,2,6,7],[3.0,7.0,1.0,178.0]) |115 |
(9,[0,2,6,7],[3.0,7.0,1.0,199.0]) |101 |
(9,[0,2,6,7],[3.0,7.0,1.0,206.0]) |133 |
(9,[0,2,6,7],[3.0,8.0,1.0,220.0]) |84 |
(9,[0,2,6,7],[4.0,9.0,1.0,261.0]) |93 |
+----------------------------------+-----+
only showing top 20 rows

In [19]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol ='features', labelCol='label', maxIter=1000, regParam=0.8, elasticNetParam=1)
lr_model = lr.fit(trainData)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [35.1740391753,91.035953909,-2.86162495611,10.1176849702,-19.9490960265,9.29596739756,-34.7054324757,0.0,0.0]
Intercept: 0.30424906816367053

In [20]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 153.822058
r2: 0.271404

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

In [22]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [23]:
layers =[4,5,4,3]

In [24]:
classifier = MultilayerPerceptronClassifier(maxIter=1000,layers=layers)

In [25]:
gridBuilder=ParamGridBuilder().addGrid(classifier.layers,[[4,2,3,3],[4,2,2,3],[4,10,3]]).addGrid(classifier.stepSize,[0.03,0.01]).build()

In [26]:
pipeLine=Pipeline()
pipeLine.setStages((classifier))

Out[ 133 ]: Pipeline_7ef0bbbe2de6

In [27]:
evaluator=MulticlassClassificationEvaluator(metricName='f1')

In [28]:
cv = CrossValidator(estimator=pipeLine,estimatorParamMaps=gridBuilder,evaluator=evaluator)

In [29]:
model = pipeLine.fit(trainData)



--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4151223953939292> in <module> () 
 ----> 1 model = pipeLine . fit ( trainData ) 
 2 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 92 def _fit ( self , dataset ) : 
 93 stages = self . getStages ( ) 
 ---> 94 for stage in stages : 
 95 if not ( isinstance ( stage , Estimator ) or isinstance ( stage , Transformer ) ) : 
 96 raise TypeError(

 TypeError : 'MultilayerPerceptronClassifier' object is not iterable

In [30]:
preds = classifier.fit(trainData)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4151223953939294> in <module> () 
 ----> 1 preds = classifier . fit ( trainData ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2890.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 117.0 failed 1 times, most recent failure: Lost task 0.0 in stage 117.0 (TID 1112, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$org$apache$spark$ml$feature$OneHotEncoderModel$$encoder$1: (double, int) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1124)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1130)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:221)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1170)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1161)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:1096)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:1161)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:883)
	at org.apache.spark.rdd.RDD.getOrCompute(RDD.scala:351)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:302)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:139)
	at or

In [31]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(maxBins=10)